In [1]:
import polars as pl
from glob import glob
from tqdm.auto import tqdm

# pip install "polars[excel]" fastexcel

In [2]:
files = glob("./MJSP/*.xlsx")
len(files)

11

In [3]:
dfs = []

for file in tqdm(files):

	df = pl.read_excel(file, read_options={
		"use_columns": ['uf', 'municipio', 'evento', 'data_referencia', 'total_vitima']
	})
	
	df = df.filter(pl.col("evento") == "Homicídio doloso")

	df = df.with_columns(
    	pl.col("total_vitima").cast(pl.Int16)
	)	

	dfs.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 10, falling back to string


In [4]:
result = pl.concat(dfs)
result.write_csv("MJSP.csv", separator=",")
print(result)

shape: (730_083, 5)
┌─────┬────────────────┬──────────────────┬─────────────────┬──────────────┐
│ uf  ┆ municipio      ┆ evento           ┆ data_referencia ┆ total_vitima │
│ --- ┆ ---            ┆ ---              ┆ ---             ┆ ---          │
│ str ┆ str            ┆ str              ┆ date            ┆ i16          │
╞═════╪════════════════╪══════════════════╪═════════════════╪══════════════╡
│ AC  ┆ ACRELÂNDIA     ┆ Homicídio doloso ┆ 2025-01-01      ┆ 0            │
│ AC  ┆ ASSIS BRASIL   ┆ Homicídio doloso ┆ 2025-01-01      ┆ 1            │
│ AC  ┆ BRASILÉIA      ┆ Homicídio doloso ┆ 2025-01-01      ┆ 0            │
│ AC  ┆ BUJARI         ┆ Homicídio doloso ┆ 2025-01-01      ┆ 0            │
│ AC  ┆ CAPIXABA       ┆ Homicídio doloso ┆ 2025-01-01      ┆ 0            │
│ …   ┆ …              ┆ …                ┆ …               ┆ …            │
│ TO  ┆ TOCANTINÓPOLIS ┆ Homicídio doloso ┆ 2015-12-01      ┆ 1            │
│ TO  ┆ TUPIRAMA       ┆ Homicídio doloso ┆ 2015-12-01  

In [10]:
df_6m = (
    result
    .with_columns([
        pl.col("data_referencia").dt.year().alias("ano"),
        (
            (pl.col("data_referencia").dt.month() - 1) // 6 + 1
        ).alias("semestre")  # 1 = jan–jun, 2 = jul–dez
    ])
    .group_by(["uf", "municipio", "ano", "semestre"])
    .agg(
        pl.col("total_vitima").sum().alias("vitimas_homicidio_doloso")
    )
    .sort(["uf", "municipio", "ano", "semestre"])
)

print(df_6m)

shape: (122_976, 5)
┌─────┬────────────┬──────┬──────────┬──────────────────────────┐
│ uf  ┆ municipio  ┆ ano  ┆ semestre ┆ vitimas_homicidio_doloso │
│ --- ┆ ---        ┆ ---  ┆ ---      ┆ ---                      │
│ str ┆ str        ┆ i32  ┆ i8       ┆ i64                      │
╞═════╪════════════╪══════╪══════════╪══════════════════════════╡
│ AC  ┆ ACRELÂNDIA ┆ 2015 ┆ 1        ┆ 1                        │
│ AC  ┆ ACRELÂNDIA ┆ 2015 ┆ 2        ┆ 0                        │
│ AC  ┆ ACRELÂNDIA ┆ 2016 ┆ 1        ┆ 3                        │
│ AC  ┆ ACRELÂNDIA ┆ 2016 ┆ 2        ┆ 3                        │
│ AC  ┆ ACRELÂNDIA ┆ 2017 ┆ 1        ┆ 3                        │
│ …   ┆ …          ┆ …    ┆ …        ┆ …                        │
│ TO  ┆ XAMBIOÁ    ┆ 2023 ┆ 2        ┆ 3                        │
│ TO  ┆ XAMBIOÁ    ┆ 2024 ┆ 1        ┆ 1                        │
│ TO  ┆ XAMBIOÁ    ┆ 2024 ┆ 2        ┆ 1                        │
│ TO  ┆ XAMBIOÁ    ┆ 2025 ┆ 1        ┆ 2                

In [5]:
hab = pl.read_csv("SIDRA_Habitantes.csv")
print(hab)

shape: (5_571, 4)
┌─────────────────────────────────┬─────────┬───────────────────────┬─────────┐
│ Nivel                           ┆ Codigo  ┆ Nome                  ┆ Num Hab │
│ ---                             ┆ ---     ┆ ---                   ┆ ---     │
│ str                             ┆ i64     ┆ str                   ┆ i64     │
╞═════════════════════════════════╪═════════╪═══════════════════════╪═════════╡
│ MU                              ┆ 1100015 ┆ Alta Floresta D'Oeste ┆ 21494   │
│ MU                              ┆ 1100023 ┆ Ariquemes             ┆ 96833   │
│ MU                              ┆ 1100031 ┆ Cabixi                ┆ 5351    │
│ MU                              ┆ 1100049 ┆ Cacoal                ┆ 86887   │
│ MU                              ┆ 1100056 ┆ Cerejeiras            ┆ 15890   │
│ …                               ┆ …       ┆ …                     ┆ …       │
│ MU                              ┆ 5222054 ┆ Vicentinópolis        ┆ 8768    │
│ MU                  

In [14]:
import polars as pl

UF_MAP = {
    11: "RO", 12: "AC", 13: "AM", 14: "RR", 15: "PA", 16: "AP", 17: "TO",
    21: "MA", 22: "PI", 23: "CE", 24: "RN", 25: "PB", 26: "PE", 27: "AL",
    28: "SE", 29: "BA",
    31: "MG", 32: "ES", 33: "RJ", 35: "SP",
    41: "PR", 42: "SC", 43: "RS",
    50: "MS", 51: "MT", 52: "GO", 53: "DF",
}

# 1) DataFrame auxiliar código_uf -> UF
uf_map_df = pl.DataFrame(
    {
        "codigo_uf": list(UF_MAP.keys()),
        "uf": list(UF_MAP.values()),
    }
)

# 2) Base de população (Censo)
hab_limpo = (
    hab
    .filter(pl.col("Nivel") == "MU")
    .select([
        pl.col("Codigo").alias("codigo_municipio"),
        pl.col("Nome").alias("municipio_nome"),
        pl.col("Num Hab").alias("populacao"),
    ])
    .with_columns([
        # prefixo da UF (primeiros 2 dígitos do código IBGE)
        (pl.col("codigo_municipio") // 100_000).alias("codigo_uf"),  # <--- AQUI
    ])
    .join(uf_map_df, on="codigo_uf", how="left")
    .with_columns([
        pl.col("municipio_nome")
          .str.to_uppercase()
          .str.strip_chars()
          .alias("municipio_norm"),
    ])
)

# 3) Base de homicídios (já agregada em df_6m)
homi_norm = (
    df_6m
    .with_columns([
        pl.col("municipio")
          .str.to_uppercase()
          .str.strip_chars()
          .alias("municipio_norm")
    ])
)

# 4) Join final: uf + nome normalizado
tabela = (
    homi_norm
    .join(
        hab_limpo,
        on=["uf", "municipio_norm"],
        how="left",
    )
    .with_columns([
        (
            pl.col("vitimas_homicidio_doloso") / pl.col("populacao") * 100_000
        ).alias("taxa_homicidios_100k")
    ])
    .select([
        "municipio_norm",
        "codigo_municipio",
        "uf",
        "ano",
        "semestre",
        "vitimas_homicidios_100k" if "vitimas_homicidios_100k" in homi_norm.columns else "vitimas_homicidio_doloso",
        "populacao",
        "taxa_homicidios_100k",
    ])
    .sort(["uf", "municipio_norm", "ano", "semestre"])
)

print(tabela)


shape: (122_976, 8)
┌───────────────┬───────────────┬─────┬──────┬──────────┬───────────────┬───────────┬──────────────┐
│ municipio_nor ┆ codigo_munici ┆ uf  ┆ ano  ┆ semestre ┆ vitimas_homic ┆ populacao ┆ taxa_homicid │
│ m             ┆ pio           ┆ --- ┆ ---  ┆ ---      ┆ idio_doloso   ┆ ---       ┆ ios_100k     │
│ ---           ┆ ---           ┆ str ┆ i32  ┆ i8       ┆ ---           ┆ i64       ┆ ---          │
│ str           ┆ i64           ┆     ┆      ┆          ┆ i64           ┆           ┆ f64          │
╞═══════════════╪═══════════════╪═════╪══════╪══════════╪═══════════════╪═══════════╪══════════════╡
│ ACRELÂNDIA    ┆ 1200013       ┆ AC  ┆ 2015 ┆ 1        ┆ 1             ┆ 14021     ┆ 7.132159     │
│ ACRELÂNDIA    ┆ 1200013       ┆ AC  ┆ 2015 ┆ 2        ┆ 0             ┆ 14021     ┆ 0.0          │
│ ACRELÂNDIA    ┆ 1200013       ┆ AC  ┆ 2016 ┆ 1        ┆ 3             ┆ 14021     ┆ 21.396477    │
│ ACRELÂNDIA    ┆ 1200013       ┆ AC  ┆ 2016 ┆ 2        ┆ 3            

In [15]:
tabela.write_csv("criminal_indicator.csv")

In [12]:
print(
    tabela
    .filter(pl.col("municipio_norm").str.contains("CAMPESTRE"))
    .select(["uf", "municipio_norm", "populacao"])
    .unique()
    .sort(["municipio_norm", "uf"])
)


shape: (6, 3)
┌─────┬───────────────────────┬───────────┐
│ uf  ┆ municipio_norm        ┆ populacao │
│ --- ┆ ---                   ┆ ---       │
│ str ┆ str                   ┆ i64       │
╞═════╪═══════════════════════╪═══════════╡
│ AL  ┆ CAMPESTRE             ┆ null      │
│ MG  ┆ CAMPESTRE             ┆ null      │
│ RS  ┆ CAMPESTRE DA SERRA    ┆ null      │
│ GO  ┆ CAMPESTRE DE GOIÁS    ┆ null      │
│ MA  ┆ CAMPESTRE DO MARANHÃO ┆ null      │
│ RN  ┆ SÃO JOSÉ DO CAMPESTRE ┆ null      │
└─────┴───────────────────────┴───────────┘
